In [1]:
import pandas as pd
import openai
import requests

#######################################
# Set your API key and data path
PATH = "./DATA/"

## OpenAI API Key
# valid for this education session only
api_key = "[API_KEY]"

# Data download from github and save to PATH
!wget -P $PATH https://raw.githubusercontent.com/leelabsg/KOGO_OpenAI_API_Tutorial_2025/main/Session1/note1_ShorthandStyle.txt
!wget -P $PATH https://raw.githubusercontent.com/leelabsg/KOGO_OpenAI_API_Tutorial_2025/main/Session1/note2_MIMICstyle.txt


--2025-09-26 01:08:44--  https://raw.githubusercontent.com/leelabsg/KOGO_OpenAI_API_Tutorial_2025/main/Session1/note1_ShorthandStyle.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 536 [text/plain]
Saving to: ‘./DATA/note1_ShorthandStyle.txt’

note1_ShorthandStyl 100%[===================>]     536  --.-KB/s    in 0s      

2025-09-26 01:08:44 (19.5 MB/s) - ‘./DATA/note1_ShorthandStyle.txt’ saved [536/536]

--2025-09-26 01:08:44--  https://raw.githubusercontent.com/leelabsg/KOGO_OpenAI_API_Tutorial_2025/main/Session1/note2_MIMICstyle.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connect

## Check Clinical Note Examples

**1) Clinical Note 불러 오는 방법 안내**

파일에서 Clinical Note 불러오기

- 텍스트(.txt) 파일에 작성된 노트를 불러오는 방식입니다.
- 불러올 파일 경로를 정확히 지정해야 합니다.


**2) 예시로 사용되는 note1과 note2는 서로 다른 내용입니다:**
- note1: shorthand 스타일의 간단한 요약형 노트
- note2: narrative 스타일의 서술형 노트로, 보다 상세한 문맥과 진단 경과가 포함되어 있습니다

이 두 노트를 각각 사용하여 prompt 차이, 언어 차이, 노트 형식 차이에 따른 분석 결과 차이를 실험해볼 수 있습니다.

### 1) Shorthand version

In [2]:
## (1) Load note from .txt file
with open(f"{PATH}note1_ShorthandStyle.txt", "r", encoding="utf-8") as f:
    note1 = f.read()

In [3]:
## check loaded note
print(note1)

#. T2DM  
   Dx since 2010, recent HbA1c 9.1  
   MTF → 설사, 복부팽만, 식욕저하, 무기력감  
   Lactic acidosis (Lac 3.9) 의심 → 약 중단, 수액 치료  
   MTF-induced lactic acidosis 의심 → 약 중단 후 회복, basal insulin 예정

   Levemir 22 --- 10 units, 저녁 인슐린 잘 안 맞음


#. HTN  
   BP on admission 92/58
   Losartan 유지 

#. CKD stage 3  
   Baseline Cr 1.6 → 입원 시 Cr 2.1  
   수액 이후 신기능 호전, 투석 필요 없음  

#. Dyslipidemia  
   Atorvastatin  



### 2) Narrative version (MIMIC-style)

In [4]:
## (1) Load note from .txt file
with open(f"{PATH}note2_MIMICstyle.txt", "r", encoding="utf-8") as f:
    note2 = f.read()

In [5]:
## check loaded note
print(note2[:500])  # 앞 500자만 출력

History of Present Illness:  
78-year-old female with a history of type 2 diabetes, hypertension, and CKD stage 3 who presented with generalized weakness, poor oral intake, and nausea.

Two weeks prior to admission, she was started on metformin after a primary care visit revealed HbA1c of 9.1. Since initiation, the patient has developed persistent loose stools and intermittent abdominal cramping. No reported fever or vomiting. Her family noted a gradual decline in appetite and energy over the pa


### 본 실습에서는 note1 (shorthand version)을 사용합니다. 만약 narrative version을 테스트하고 싶으면 아래의 코드를 수정하면 됩니다.

In [6]:
note = note1

## 1. Setting OpenAI API

In [7]:
client = openai.OpenAI(api_key=api_key)

## [Task 1] Drug Name Extraction and drug code identification

이 Task에서는 임상노트로부터 언급된 약물명을 추출하는 작업을 수행합니다.

- GPT에게 임상노트를 입력하고, 텍스트에서 **약물명** 추출 및 약물에 상응하는 **ATC CODE**를 추출하게 합니다.
- 약물은 **일반명** 또는 **약어**(예: MFM = metformin)로 표현될 수 있습니다.



### (1) Drug name only

In [19]:
prompt1_eng = f"""

Task: Extract all drug names mentioned in the following clinical note.
- Drug names may be written as abbreviations.
- Return the list of drug names only, without any additional explanation.

Clinical Note:
{note}
"""

In [23]:
## English Prompt Version
model="gpt-5" # select model

response = client.responses.create(
    model=model,
    instructions="You are a clinical text extraction assistant.",
    input=prompt1_eng
)

print(response.output_text)

MTF
insulin
Levemir
Losartan
Atorvastatin


### (2) Drug name+description+code

In [24]:
prompt2_eng = f"""

Task: Extract all drug names mentioned in the following clinical note.
- Drug names may be written as abbreviations.
- Return the list of drug names, simple descriptions, and corresponding ATC codes in a table format (CSV).

Clinical Note:
{note}
"""

response = client.responses.create(
    model=model,
    instructions="You are a clinical text extraction assistant.",
    input=prompt2_eng
)

print(response.output_text)

Drug name,Description,ATC code
MTF (metformin),Biguanide oral antihyperglycemic for type 2 diabetes,A10BA02
Levemir (insulin detemir),Long-acting/basal insulin analog,A10AE05
Losartan,Angiotensin II receptor blocker antihypertensive,C09CA01
Atorvastatin,HMG-CoA reductase inhibitor (statin) for dyslipidemia,C10AA05


### (3) Usage check
사용한 토큰을 아래의 코드를 이용해서 체크할 수 있습니다.

In [25]:
print(response.usage)

ResponseUsage(input_tokens=251, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=1319, output_tokens_details=OutputTokensDetails(reasoning_tokens=1216), total_tokens=1570)


## [Task 2] Adverse Drug Reaction Detection

이 Task에서는 임상노트에 언급된 **약물 부작용(Adverse Drug Reactions, ADR)** 을 탐지하는 작업을 수행합니다.

- GPT에게 임상노트를 입력하고, 특정 약물 투여 이후 발생한 부작용을 **약물-증상 쌍(drug → symptom)** 형태로 추출하게 합니다.
- 약물명은 약어로 표현될 수 있으며, 부작용은 명시적 또는 암시적으로 표현되어 있을 수 있습니다.




### (1) ADR

In [26]:
## Prompt
prompt2_adr = f"""

Task: Extract all possible adverse drug reactions (ADRs) mentioned in the clinical note.
- Return results in JSON format, with each item containing:
  - "drug": drug name
  - "symptom": symptom or adverse effect
  - "evidence": supporting sentence or phrase from the note
- No extra explanation.

Clinical Note:
{note}
"""

In [27]:

response = client.responses.create(
    model=model,
    instructions="You are a clinical text extraction assistant.",
    input=prompt2_adr
)

print(response.output_text)

[
  {
    "drug": "MTF",
    "symptom": "설사",
    "evidence": "MTF → 설사, 복부팽만, 식욕저하, 무기력감"
  },
  {
    "drug": "MTF",
    "symptom": "복부팽만",
    "evidence": "MTF → 설사, 복부팽만, 식욕저하, 무기력감"
  },
  {
    "drug": "MTF",
    "symptom": "식욕저하",
    "evidence": "MTF → 설사, 복부팽만, 식욕저하, 무기력감"
  },
  {
    "drug": "MTF",
    "symptom": "무기력감",
    "evidence": "MTF → 설사, 복부팽만, 식욕저하, 무기력감"
  },
  {
    "drug": "MTF",
    "symptom": "lactic acidosis",
    "evidence": "MTF-induced lactic acidosis 의심 → 약 중단 후 회복, basal insulin 예정"
  }
]


### (2) Verification Agent

위의 LLM이 생성한 답변이 맞는지 검증하기 위한 Agent를 생성해서 답변을 검증합니다.

In [28]:

verifier_prompt = """
If you disagree, provide counterarguments.
Provide agree and comment for each drug and side effect pairs.
Respond in JSON:
{"agree": true/false, "comments": "..."}.

"""

extractor_output = response.output_text
verifier_input=f"{verifier_prompt}\n\nExtractorAgent output:\n{extractor_output}\n\nMedical note:\n{note}"

model="gpt-5-mini" # change model

verifier_response = client.responses.create(
    model=model,
    instructions="You are VerifierAgent. Your job is to check ExtractorAgent's identified side effects.",
    input=verifier_input
)

print(verifier_response.output_text)


[
  {
    "drug": "MTF",
    "symptom": "설사",
    "agree": true,
    "comments": "Agree. The medical note explicitly lists '설사' after MTF and diarrhea is a well-known, common gastrointestinal adverse effect of metformin."
  },
  {
    "drug": "MTF",
    "symptom": "복부팽만",
    "agree": true,
    "comments": "Agree. '복부팽만' (abdominal bloating) is documented in the note and can occur with metformin as part of GI intolerance."
  },
  {
    "drug": "MTF",
    "symptom": "식욕저하",
    "agree": true,
    "comments": "Agree. Decreased appetite is recorded in the note and is a recognized, albeit less specific, side effect associated with metformin use."
  },
  {
    "drug": "MTF",
    "symptom": "무기력감",
    "agree": true,
    "comments": "Agree, with caution. The note lists '무기력감' (fatigue/lethargy). Fatigue is nonspecific and could be due to many causes (infection, uremia from CKD/AKI, metabolic disturbance, or lactic acidosis), so although it is attributed here to MTF, the association is less s

## [Task 3] Clinical Note Summarization

이 Task에서는 임상노트에 포함된 핵심 정보를 GPT를 통해 한글로 **요약(summarization)** 하는 작업을 수행합니다.
- 임상노트에는 환자의 병력, 진단, 투약, 검사 결과, 치료 계획 등의 정보가 포함되어 있으며, 종종 매우 길고 복잡합니다.
- 이 Task는 환자에 대한 중요한 정보를 요약하여, 환자/보호자 설명에 활용될 수 있는 간결한 요약문을 생성하는 것이 목적입니다.
- 출력은 한국어 요약문으로 생성됩니다.



In [29]:
prompt3_family = f"""
Task: Summarize the following clinical note in simple language for the patient’s family.
- Use non-technical, plain Korean language.
- The summary should explain what happened, what was diagnosed, how it was treated, and what to expect.
- Avoid complex terminology and abbreviations.

Output language: Korean

Clinical Note:
{note2}
"""

In [30]:
## Family-Friendly Summary
model = "gpt-5" # select model

response = client.responses.create(
    model=model,
    instructions="You are a clinical text summarization assistant.",
    input=prompt3_family
)

print(response.output_text)

가족분들을 위한 요약

무슨 일이 있었나요?
- 어르신은 원래 당뇨병, 혈압, 콩팥 기능 저하가 있습니다.
- 2주 전 당 조절을 위해 새로 시작한 당뇨약(메트포르민)을 드신 뒤부터 설사가 계속되고, 배가 짧게 아픈 일이 있었으며, 식욕이 떨어지고 기운이 없어졌습니다.
- 병원에 왔을 때 혈압이 많이 낮고 맥이 빨랐습니다. 피검사에서 콩팥 기능 수치가 악화되어 있었고, 몸이 산성 쪽으로 기운 상태(몸에 산성 물질이 쌓인 상태)로 나왔습니다. 감염을 의심할 만한 증거는 없었습니다.

무엇으로 진단했나요?
- 최근 시작한 메트포르민이, 원래 콩팥 기능이 약한 상황과 겹쳐서, 탈수와 함께 몸의 산성도가 올라간 것으로 의심했습니다.
- 탈수로 인해 혈압이 낮아진 것도 함께 있었습니다.

어떻게 치료했나요?
- 메트포르민은 즉시 중단했습니다. 앞으로도 다시 드시면 안 됩니다.
- 정맥으로 수액을 충분히 주었고, 산성도를 낮추는 약을 함께 사용했습니다.
- 48시간 동안 지켜보는 동안 혈압과 전신 상태가 점차 좋아졌고, 콩팥 수치도 호전되었습니다. 투석은 필요하지 않았습니다.
- 당 조절을 위해 먹는 당뇨약은 잠시 모두 중단했고, 피당 수치를 보며 밤에 맞는 소량의 인슐린으로 조절을 시작하기로 했습니다.
- 가슴 엑스레이와 콩팥 초음파는 큰 이상이 없었습니다. 소변 검사에서 감염 소견은 없었습니다.

현재 상태와 앞으로의 계획
- 전반적으로 많이 안정되었습니다. 수액 치료 후 혈압과 컨디션이 좋아졌고, 콩팥 기능도 회복 중입니다.
- 퇴원 후에는 원래 지내시던 시설로 돌아가시며, 메트포르민은 앞으로 절대 드시면 안 됩니다.
- 혈당은 하루 두 번 체크하고, 결과에 따라 인슐린 용량을 천천히 조절할 예정입니다.
- 콩팥 전문의와 당뇨 전문의 진료를 1–2주 안에 받도록 예약합니다.
- 응급 상황이 생기면 심폐소생술을 포함한 모든 치료를 받기로 결정되어 있습니다.

퇴원 약 안내(쉽게)
- 밤에 맞는 인슐린 6단위: 당 조절을 위해 사용합니다.
- 로사르탄 50mg 하루 1